<a href="https://colab.research.google.com/github/piyushy1/Gradientor/blob/main/Dirichlet_Distribution/nn_hyperparam_tuning_dirichlet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>